In [2]:
#Preparation
%load_exp autoreload
%autoreaload 2
import os
os.chdir('/home/dmitryhd/repos/vacancy_analysis')
#print(os.getcwd())
import vacan.processor.features as ft
import vacan.processor.data_model as dm
from vacan.features_config import FEATURES as ft_config

In [3]:
#Load raw vacs
dbmanager = dm.DBManager('vacan')
print('Getting raw vacancies...')
raw_vacs = dbmanager.get_raw_vacs()
print('Got {} raw vacancies from db.'.format(len(raw_vacs)))

Getting raw vacancies...
Got 65696 raw vacancies from db.


In [4]:
#Process vacancies
qualifier = ft.Qualifier(ft_config)
processor = ft.VacancyProcessor(qualifier)
proc_vacs = processor.process(raw_vacs)
print('Got %i processed vacancies.' % len(proc_vacs))

Got 5755 processed vacancies.


In [ ]:
salary[]